# Autoencoders Variacionales

In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F

# Defino una versión muy simple de un VAE para que podamos ver sus componentes.
class ToyVAE(nn.Module):
    def __init__(self, input_dim=35, latent_dim=2):
        super(ToyVAE, self).__init__()
        # Mi encoder tiene una capa que va a los parámetros de la distribución.
        self.fc_encode = nn.Linear(input_dim, 20)
        self.fc_mu = nn.Linear(20, latent_dim)      # Capa para la media
        self.fc_log_var = nn.Linear(20, latent_dim) # Capa para la log-varianza

        # Mi decoder tiene una capa para volver al espacio original.
        self.fc_decode = nn.Linear(latent_dim, 20)
        self.fc_output = nn.Linear(20, input_dim)

    def encode(self, x):
        h = F.relu(self.fc_encode(x))
        return self.fc_mu(h), self.fc_log_var(h)

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc_decode(z))
        return torch.sigmoid(self.fc_output(h))

    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        recon_x = self.decode(z)
        return recon_x, mu, log_var

#  la función de pérdida del VAE.
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE, KLD

#  instancia de mi modelo y un optimizador.
model = ToyVAE()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Creo mi único dato de entrada: una imagen de un "7" en formato 7x5 (35 píxeles).
# El 1.0 representa un píxel encendido, el 0.0 uno apagado.
X = torch.tensor([
    1.0, 1.0, 1.0, 1.0, 0.0,
    0.0, 0.0, 0.0, 1.0, 0.0,
    0.0, 0.0, 1.0, 0.0, 0.0,
    0.0, 1.0, 0.0, 0.0, 0.0,
    0.0, 1.0, 0.0, 0.0, 0.0,
    0.0, 1.0, 0.0, 0.0, 0.0,
    0.0, 1.0, 0.0, 0.0, 0.0
])

print("--- INICIO DE LA DEMOSTRACIÓN DE UN PASO DE ENTRENAMIENTO ---")
print(f"Dato de entrada X (un '7' de 35 píxeles) definido.\n")

# --- PASO 1: Generar Predicciones (Forward Pass) ---
print("--- [Paso 1] Generando Predicciones ---")
# Hago pasar mi dato 'X' a través del modelo.
recon_X_hat, mu, log_var = model(X)

print(f"1a. Encoder produce parámetros de la 'nube':")
print(f"    - Media (mu): {mu.detach().numpy().round(2)}")
print(f"    - Log-Varianza (log_var): {log_var.detach().numpy().round(2)}")

# El vector latente z se muestrea usando la reparametrización.
z = model.reparameterize(mu, log_var)
print(f"\n1b. Se muestrea un vector latente 'z' de la nube: {z.detach().numpy().round(2)}")

print(f"\n1c. Decoder genera la reconstrucción X_hat a partir de 'z'.")
print(f"    (Mostrando los primeros 10 de 35 píxeles de X_hat: {recon_X_hat.detach().numpy()[:10].round(2)})")


# --- PASO 2: Calcular Pérdida ---
print("\n--- [Paso 2] Calculando la Pérdida Dual ---")
loss_recon, loss_kl = loss_function(recon_X_hat, X, mu, log_var)
total_loss = loss_recon + loss_kl

print(f"2a. Pérdida de Reconstrucción (qué tan diferente es X de X_hat): {loss_recon.item():.2f}")
print(f"2b. Pérdida de Regularización KL (qué tan 'desorganizada' está la nube): {loss_kl.item():.2f}")
print("--------------------------------------------------")
print(f"PÉRDIDA TOTAL (Suma de ambas): {total_loss.item():.2f}")


# --- PASO 3: Ajustar Parámetros ---
print("\n--- [Paso 3] Ajustando Parámetros del Modelo ---")
# Primero, veo un peso del modelo ANTES de la actualización.
weight_antes = model.fc_output.weight.data[0, 0].item()
print(f"Peso de ejemplo ANTES de la actualización: {weight_antes:.4f}")

# Calculo los gradientes para todos los parámetros con respecto a la pérdida total.
total_loss.backward()

# El optimizador usa los gradientes para actualizar los pesos.
optimizer.step()

# Reviso el mismo peso DESPUÉS de la actualización.
weight_despues = model.fc_output.weight.data[0, 0].item()
print(f"Peso de ejemplo DESPUÉS de la actualización: {weight_despues:.4f}")
print("¡El peso ha cambiado! El modelo está aprendiendo.")


# --- PASO 4: Y Repetir ---
print("\n--- [Paso 4] Repitiendo el Proceso ---")
print("Ahora, el ciclo completo se repetiría con nuevos datos (u los mismos).")
print("Veamos cómo cambia la pérdida si repetimos el proceso 5 veces más con el mismo dato:")

for i in range(5):
    # Paso 1 y 2
    recon_X_hat, mu, log_var = model(X)
    loss_recon, loss_kl = loss_function(recon_X_hat, X, mu, log_var)
    total_loss = loss_recon + loss_kl

    # Paso 3
    optimizer.zero_grad() # reseteo los gradientes antes de volver a calcularlos.
    total_loss.backward()
    optimizer.step()

    print(f"  Iteración {i+1}: Pérdida Total = {total_loss.item():.2f} (Recon: {loss_recon.item():.2f}, KL: {loss_kl.item():.2f})")

print("\nComo se puede ver, la pérdida total tiende a disminuir a medida que el modelo se ajusta.")
print("--- FIN DE LA DEMOSTRACIÓN ---")

--- INICIO DE LA DEMOSTRACIÓN DE UN PASO DE ENTRENAMIENTO ---
Dato de entrada X (un '7' de 35 píxeles) definido.

--- [Paso 1] Generando Predicciones ---
1a. Encoder produce parámetros de la 'nube':
    - Media (mu): [-0.15  0.02]
    - Log-Varianza (log_var): [-0.11  0.16]

1b. Se muestrea un vector latente 'z' de la nube: [ 0.77 -0.24]

1c. Decoder genera la reconstrucción X_hat a partir de 'z'.
    (Mostrando los primeros 10 de 35 píxeles de X_hat: [0.51 0.59 0.56 0.47 0.54 0.48 0.41 0.58 0.47 0.5 ])

--- [Paso 2] Calculando la Pérdida Dual ---
2a. Pérdida de Reconstrucción (qué tan diferente es X de X_hat): 23.67
2b. Pérdida de Regularización KL (qué tan 'desorganizada' está la nube): 0.02
--------------------------------------------------
PÉRDIDA TOTAL (Suma de ambas): 23.69

--- [Paso 3] Ajustando Parámetros del Modelo ---
Peso de ejemplo ANTES de la actualización: -0.2202
Peso de ejemplo DESPUÉS de la actualización: -0.2202
¡El peso ha cambiado! El modelo está aprendiendo.

--- 

## GAN

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os

# --- PASO 1: Mis Notas de Configuración ---
# Defino algunos parámetros clave.
latent_dim = 100      # Dimensión del ruido de entrada. Es como el tamaño del "lienzo" para el generador.
batch_size = 64       # Cuántas imágenes procesar a la vez.
epochs = 25           # Cuántas vueltas completas dar al conjunto de datos.
learning_rate = 0.0002 # Qué tan grandes son los pasos que da el optimizador para aprender.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Usar la GPU si está disponible.

# Me aseguro de tener una carpeta para guardar las imágenes que genere.
# Así puedo ver cómo mi falsificador va mejorando con el tiempo.
if not os.path.exists('gan_basica_results'):
    os.makedirs('gan_basica_results')

# --- PASO 2: Preparar los Datos Reales ---
# Necesito los datos reales para enseñarle al detective cómo son los dígitos de verdad.
# Voy a usar el famoso conjunto de datos MNIST.
transform = transforms.Compose([
    transforms.ToTensor(), # Convierto las imágenes a tensores de PyTorch.
    transforms.Normalize([0.5], [0.5]) # Normalizo los píxeles al rango [-1, 1]. Esto es importante para que la función Tanh del generador funcione bien.
])

mnist_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
data_loader = DataLoader(dataset=mnist_data, batch_size=batch_size, shuffle=True)

# --- PASO 3: Construir mis Redes Neuronales ---

# Primero, el Falsificador (El Generador).
# Su trabajo es tomar ruido aleatorio y convertirlo en algo que parezca un dígito.
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        # Mi generador va a ser una red neuronal simple, con capas lineales.
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 784), # 784 porque las imágenes son de 28x28 píxeles.
            nn.Tanh() # La función Tanh asegura que la salida esté en el rango [-1, 1].
        )

    def forward(self, z):
        # Tomo el ruido 'z' y lo paso por el modelo.
        img = self.model(z)
        # Le doy la forma correcta a la salida (28x28).
        img = img.view(img.size(0), 1, 28, 28)
        return img

# Ahora, el Detective (El Discriminador).
# Su trabajo es tomar una imagen y dar una probabilidad de que sea real.
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid() # La sigmoide da una probabilidad entre 0 (falsa) y 1 (real).
        )

    def forward(self, img):
        # Primero aplano la imagen para que entre en la red.
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

# --- PASO 4: Poner Todo en Marcha ---
# Creo instancias de mis dos redes.
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Defino la función de pérdida. La Entropía Cruzada Binaria es perfecta para este juego de clasificación.
loss_fn = nn.BCELoss()

# Creo optimizadores separados para cada red. Esto es clave.
optimizer_G = torch.optim.Adam(generator.parameters(), lr=learning_rate)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=learning_rate)

print("Iniciando el entrenamiento de la GAN...")

# --- PASO 5: El Ciclo de Entrenamiento ---
# Aquí es donde ocurre la magia (y la batalla).
for epoch in range(epochs):
    for i, (real_imgs, _) in enumerate(data_loader):

        # Preparo las etiquetas objetivo: 1 para imágenes reales, 0 para falsas.
        real_labels = torch.ones(real_imgs.size(0), 1).to(device)
        fake_labels = torch.zeros(real_imgs.size(0), 1).to(device)

        real_imgs = real_imgs.to(device)

        # --- Fase 1: Entrenar al Detective (Discriminador) ---
        # El objetivo es que mejore en su trabajo de distinguir.
        optimizer_D.zero_grad()

        # Pérdida con imágenes reales: ¿qué tan bien las identifica como reales?
        d_real_output = discriminator(real_imgs)
        d_real_loss = loss_fn(d_real_output, real_labels)

        # Genero un lote de imágenes falsas.
        z_noise = torch.randn(real_imgs.size(0), latent_dim).to(device)
        fake_imgs = generator(z_noise)

        # Pérdida con imágenes falsas: ¿qué tan bien las identifica como falsas?
        # Uso .detach() para que los gradientes no afecten al generador en este paso.
        d_fake_output = discriminator(fake_imgs.detach())
        d_fake_loss = loss_fn(d_fake_output, fake_labels)

        # La pérdida total del detective es el promedio de ambas.
        d_loss = (d_real_loss + d_fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        # --- Fase 2: Entrenar al Falsificador (Generador) ---
        # El objetivo es que mejore engañando al detective.
        optimizer_G.zero_grad()

        # Genero un nuevo lote de imágenes falsas.
        gen_imgs = generator(z_noise)

        # Calculo la pérdida del generador. El generador GANA si el detective
        # piensa que sus imágenes falsas son REALES (etiqueta 1).
        g_loss = loss_fn(discriminator(gen_imgs), real_labels)

        g_loss.backward()
        optimizer_G.step()

    # Imprimo el progreso al final de cada época para ver cómo van las pérdidas.
    print(f"[Época {epoch+1}/{epochs}] [Pérdida D: {d_loss.item():.4f}] [Pérdida G: {g_loss.item():.4f}]")

    # Guardo una muestra de las imágenes generadas en esta época para ver el progreso visualmente.
    save_image(gen_imgs.data[:25], f"gan_basica_results/epoch_{epoch+1}.png", nrow=5, normalize=True)

print("Entrenamiento finalizado. Revisa la carpeta 'gan_basica_results' para ver las imágenes generadas.")

100%|██████████| 9.91M/9.91M [00:25<00:00, 391kB/s] 
100%|██████████| 28.9k/28.9k [00:00<00:00, 143kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.46MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 1.58MB/s]


Iniciando el entrenamiento de la GAN...
[Época 1/25] [Pérdida D: 0.3028] [Pérdida G: 3.0886]
[Época 2/25] [Pérdida D: 0.2147] [Pérdida G: 2.3709]
[Época 3/25] [Pérdida D: 0.6653] [Pérdida G: 1.1661]
[Época 4/25] [Pérdida D: 0.3138] [Pérdida G: 3.0648]
[Época 5/25] [Pérdida D: 0.2672] [Pérdida G: 2.3449]
[Época 6/25] [Pérdida D: 0.4606] [Pérdida G: 1.6277]
[Época 7/25] [Pérdida D: 0.4339] [Pérdida G: 2.0970]
[Época 8/25] [Pérdida D: 0.6758] [Pérdida G: 0.9634]
[Época 9/25] [Pérdida D: 0.2383] [Pérdida G: 2.2362]
[Época 10/25] [Pérdida D: 0.2985] [Pérdida G: 2.2868]
[Época 11/25] [Pérdida D: 0.2392] [Pérdida G: 3.0469]
[Época 12/25] [Pérdida D: 0.1661] [Pérdida G: 2.2942]
[Época 13/25] [Pérdida D: 0.1168] [Pérdida G: 2.8628]
[Época 14/25] [Pérdida D: 0.1212] [Pérdida G: 3.7772]
[Época 15/25] [Pérdida D: 0.1394] [Pérdida G: 3.8864]
[Época 16/25] [Pérdida D: 0.2688] [Pérdida G: 2.9559]
[Época 17/25] [Pérdida D: 0.3876] [Pérdida G: 2.8976]
[Época 18/25] [Pérdida D: 0.2652] [Pérdida G: 2.758